In [1]:
import math
%matplotlib inline
import matplotlib.pyplot as plt
from IPython import display
from IPython.display import Javascript
import time
import numpy as np
import matplotlib.patches as patches
import ipywidgets as widgets

plt.rc({'font.size': 22})


gamma_slider = widgets.FloatSlider(
    value=150,
    min=0,
    max=750,
    step=1.0,
    description='$\gamma$ (pg/s):',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.0f',
)
    
temp_slider = widgets.FloatSlider(
    value=0,
    min=0,
    max=1000,
    step=1.0,
    description='T (K):',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.0f',
)

emax_slider = widgets.FloatSlider(
    value=100,
    min=0,
    max=250,
    step=1.0,
    description='$E_{max}\cdot 10^{-4}$ (statV/cm):',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.0f',
)

tmax_slider = widgets.FloatSlider(
    value=500,
    min=0,
    max=3000,
    step=1.0,
    description='$t_{max}$ (fs)',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.0f',
)

tau2max_slider = widgets.FloatSlider(
    value=0,
    min=0,
    max=0,
    step=1,
    description='$\\tau_{2}^{(max)}$ (fs)',
    disabled=True,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

vpulse_slider = widgets.FloatSlider(
    value=50.0,
    min=0,
    max=100,
    step=0.1,
    description='$\\nu_{pulse}$ (THz)',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

nmol_slider = widgets.IntSlider(
    value=1,
    min=1,
    max=1000,
    step=1,
    description='N$_{mol}$',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
)


tauswitch_slider = widgets.IntSlider(
    min=1,
    value=1000,
    max=1000,
    step=1,
    description='$\\tau_{switch}$ (fs)',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.0f',
)

disorder_cb = widgets.Checkbox(
    value=False,
    description='Include Disorder?',
    disabled=False
)

sigw_slider = widgets.IntSlider(
    min=1,
    value=10,
    max=150,
    step=1,
    description='$\\sigma_{inh}$ (cm$^{-1}$)',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.0f',
)

wcenter_slider = widgets.IntSlider(
    min=1600,
    value=1650,
    max=1700,
    step=1,
    description='$\\omega_{o}/2\\pi c$ (cm$^{-1}$)',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.0f',
)

def run_all(ev):
    global go
    go = True
    display.display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.ncells())'))

button = widgets.Button(description="Go!")
button.on_click(run_all)

go = False

pulse_box = widgets.VBox([tmax_slider, tau2max_slider, wcenter_slider, sigw_slider, disorder_cb])
part_box = widgets.VBox([nmol_slider, gamma_slider, temp_slider, tauswitch_slider])

display.display(widgets.HBox([part_box, pulse_box]))
display.display(button)

Button(description='Go!', style=ButtonStyle())

In [2]:
gamma = gamma_slider.value*1e-12  # grams/second
Temp = temp_slider.value          # K
tmax = tmax_slider.value*1e-15      # Total simulation time in seconds
Nmol = nmol_slider.value
Npos = Nmol
tau2max = tau2max_slider.value*1.0e-15
tau_switch = tauswitch_slider.value*1.0e-15
vpulse = vpulse_slider.value*1e+12                      # pulse frequency in Hz
wcenter = 6.28*wcenter_slider.value*(2.9979e+10)        # oscillator center frequency (Hz)
sigw = 6.28*sigw_slider.value*(2.9979e+10)              # SDF standard deviation (Hz)

Emax = 50e+4   # Maximum electric field in statV/cm

def calc_accel(x,efield):
    ax = - 2.0*De*alpha*(np.exp(-alpha*x) - np.exp(-2.0*alpha*x))/M
    ax += Qo*efield/M
    #ax = -K*x/M
    return ax

def vv_step(x,vx,ax,efield):
    axrand = math.sqrt(2.0*kB*Temp*gamma/dt)*np.random.normal(0,1,1)/M
    xnew = x + B*dt*vx + 0.5*B*dt*dt*(ax + axrand)
    axnew = calc_accel(xnew,efield)
    vxnew = A*vx + 0.5*dt*(A*ax + axnew + 2.0*B*axrand)
    return xnew,vxnew,axnew

def gauss_pulse(t):
    return np.cos(2.0*math.pi*(t-to)*nu)*np.exp(-((t-to)**2)/(2.0*sigma*sigma))

def morse_energy(X):
    return De*(1.0 - np.exp(-alpha*X))**2

def calc_response(Efield):
    global K
    global alpha
    global wo
    X = np.zeros((Nmol))
    VX = np.zeros((Nmol))
    AX = calc_accel(X,Efield[0])
    Mu = np.zeros((Nsteps))
    for n in range(0,Nsteps):
        X,VX,AX = vv_step(X,VX,AX,Efield[n])
        Mu[n] = np.mean(X)*Qo
        wo = np.reshape(Wo[:,n], (Npos,))
        K = M*wo**2
        alpha = np.sqrt(K/(2.0*De))
    return Mu


to = 150e-15
sigma = 20e-15
nu = 50e+12

dt=2e-15     # Time-step in seconds
Nsteps=int((2.0*tmax+tau2max+3.0*sigma*2.355)/dt)
M=12*(1.66054e-24)   # Mass in g
Qo = 4.803e-10         # Elementary charge in statCoulombs
taxis = np.arange(0,Nsteps)*dt   # Time axis (array of time steps)

Nzpad = int(1e+4)

kB = 1.38064852e-16                  # erg/K
B = 1.0/(1.0 + 0.5*gamma*dt/M)
A = B*(1.0 - 0.5*gamma*dt/M)

De = 1e+5*(kB*300.0)

# Oscillator (angular) frequency
if disorder_cb.value==False:
    Nmol = 1
    Npos = 1
    nmol_slider.value = 1
    Wo = np.ones((Npos,Nsteps))*wcenter
else:
    Wo = np.zeros((Npos,Nsteps))
    Wo[:,0] = np.random.normal(wcenter, sigw, (Npos,))

if disorder_cb.value==True:
    for n in range(1, Nsteps):
        Wo[:,n] = Wo[:,n-1]
        r = np.random.random((Npos,))
        pswitch = 2.0*dt/tau_switch
        switch_ndcs = np.where(r<pswitch)[0]
        new_freqs = np.random.normal(wcenter, sigw, np.shape(switch_ndcs))
        if len(new_freqs)>0:
            Wo[switch_ndcs,n] = new_freqs
            
wo = np.reshape(Wo[:,0], (Npos,))
K = M*wo**2
alpha = np.sqrt(K/(2.0*De))

c = 2.9979e+10
vaxis = np.fft.fftfreq(Nzpad)/(dt*c)

v1 = 1200
v2 = 2000

ndx1 = np.where(np.abs(vaxis-v1)==np.min(np.abs(vaxis-v1)))[0][0]
ndx2 = np.where(np.abs(vaxis-v2)==np.min(np.abs(vaxis-v2)))[0][0]

if go:

    tauStep1 = 6*1e-15
    tauStep2 = 4*1e-15
    Tau1 = np.arange(0.0e-15, tmax, tauStep1)
    if tau2max>0:
        Tau2 = np.arange(0.0e-15, tau2max, tauStep2)
    else:
        Tau2 = [0.0]
    Emax = 50e+4
    scfac = 1.0
    for n2 in range(0, len(Tau2)):
        tau2 = Tau2[n2]
        for n1 in range(0, len(Tau1)):
            tau1 = Tau1[n1]

            E1 = Emax*gauss_pulse(taxis+tau1+tau2-tmax-3.0*sigma*2.355)
            E2 = Emax*gauss_pulse(taxis+tau2-tmax-3.0*sigma*2.355)
            E3 = Emax*gauss_pulse(taxis-tmax-3.0*sigma*2.355)

            Mu123 = calc_response(E1 + E2 + E3)
            Mu12 = calc_response(E1 + E2)
            Mu13 = calc_response(E1 + E3)
            Mu23 = calc_response(E2 + E3)
            Mu1 = calc_response(E1)
            Mu2 = calc_response(E2)
            Mu3 = calc_response(E3)

            Mu2D = Mu123 - Mu12 - Mu23 - Mu13 + Mu1 + Mu2 + Mu3
            fMu2D = np.fft.fft(Mu2D, n=Nzpad)
            fEprobe = np.fft.fft(E3, n=Nzpad)

            Sig = -np.imag(fMu2D/fEprobe)

            if (n1==0) and (n2==0):
                scfac = 1.0/np.max(np.abs(Mu123))
                SpecMat = np.zeros((len(Tau1),len(Tau2),len(vaxis[ndx1:ndx2])))
                ppPts, = plt.plot(vaxis[ndx1:ndx2], Sig[ndx1:ndx2])
                #txt = plt.text(1750, 0.15, '$\\tau_1 = $'+str(int(tau1*1e+15))+' fs \n$\\tau_2$= '+str(int(tau2*1e+15))+' fs', fontsize=20)
                display.display(plt.gcf())
                display.clear_output(wait=True)

                ax2 = plt.axes([1.0,0.275,1,0.6])
                t3 = tmax + 3.0*sigma*2.355 + to
                field1Pts, = plt.plot((taxis-t3)*1e+15, E1/Emax, 'b')
                field2Pts, = plt.plot((taxis-t3)*1e+15, E2/Emax, 'g')
                field3Pts, = plt.plot((taxis-t3)*1e+15, E3/Emax, 'r')
                muPts, = plt.plot((taxis-t3)*1e+15, Mu123*scfac, 'y', linewidth=3)
                plt.xlabel('Time (fs)')
    #             plt.yticks([])
                plt.legend(['$E_1$', '$E_2$', '$E_3$', '$\\mu$'], fontsize=24, loc=(1.1,0.1))
            elif(n1%1==0):
                ppPts.set_ydata(Sig[ndx1:ndx2])
                #fieldPts.set_ydata((E1+E2+E3)/(3.0*Emax))
                field1Pts.set_ydata((E1)/(Emax))
                field2Pts.set_ydata((E2)/(Emax))
                field3Pts.set_ydata((E3)/(Emax))
                muPts.set_ydata(Mu123*scfac)
    #             txt.set_text('$\\tau_1 = $'+str(int(tau1*1e+15))+' fs \n$\\tau_2$= '+str(int(tau2*1e+15))+' fs')
                display.display(plt.gcf())
                display.clear_output(wait=True)
            SpecMat[n1,n2,:] = Sig[ndx1:ndx2]

    if tau2max>0:
        # Fourier transform in tau2
        fSpecMat = np.fft.fft(SpecMat, n=Nzpad, axis=1)
        vaxis_tau = np.fft.fftfreq(len(Tau2))/(c*tauStep2)
        window = (np.abs(vaxis_tau)<500)
        f2DMat = np.zeros(np.shape(fSpecMat))*1j
        for n1 in range(0, len(Tau1)):
            for n3 in range(0, len(vaxis[ndx1:ndx2])):
                f2DMat[n1,:,n3] = fSpecMat[n1,:,n3]*window
    else:
        f2DMat = SpecMat
    #Mat2D = np.real(np.fft.ifft(f2DMat, axis=1))

    # Fourier transform in Tau1
    Mat2D = np.fft.fft(f2DMat, n=Nzpad, axis=0)

    vaxis1 = np.fft.fftfreq(Nzpad)/(c*tauStep1)

    a2 = np.where(np.abs(vaxis1+v1)==np.min(np.abs(vaxis1+v1)))[0][0]
    a1 = np.where(np.abs(vaxis1+v2)==np.min(np.abs(vaxis1+v2)))[0][0]

    a1x = np.where(np.abs(vaxis1-v1)==np.min(np.abs(vaxis1-v1)))[0][0]
    a2x = np.where(np.abs(vaxis1-v2)==np.min(np.abs(vaxis1-v2)))[0][0]

    ndx1 = np.where(np.abs(vaxis-v1)==np.min(np.abs(vaxis-v1)))[0][0]
    ndx2 = np.where(np.abs(vaxis-v2)==np.min(np.abs(vaxis-v2)))[0][0]

    vaxis1 = vaxis1[a1:a2]
    vaxis1x = vaxis1[a1x:a2x]
    vaxis3 = vaxis[ndx1:ndx2]

    SigMat = Mat2D[a1:a2,0,:]
    SigMatx = Mat2D[a1x:a2x,0,:]

    plt.figure(3)
    plt.contourf(vaxis1,vaxis3,np.transpose(np.real(SigMat)) + np.transpose(np.real(SigMatx)), 25)

    plt.set_cmap(plt.cm.get_cmap('bwr'))
    plt.tick_params(axis='both', which='major', labelsize=14)
    plt.ylabel('$\\omega_3/2\pi c$', fontsize=20)
    plt.xlabel('$\\omega_1/2\pi c$', fontsize=20)

    # plt.ylim([1575,1825])
    plt.colorbar()
    plt.show()